In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

print("Path to dataset files:", path)

In [ ]:
import torch, os, json, random, numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
DATA_DIR = '/kaggle/input/chest-xray-pneumonia/chest_xray'
BATCH_SIZE = 32
IMG_SIZE = 224


class To3Channel:
    def __call__(self, img):
        if img.mode != 'RGB':
            return img.convert('RGB')
        return img

train_tfms = transforms.Compose([
    To3Channel(),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.85, 1.0)),
    transforms.RandomRotation(7),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
val_tfms = transforms.Compose([
    To3Channel(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_ds = datasets.ImageFolder(os.path.join(DATA_DIR, 'train'), transform=train_tfms)
val_ds   = datasets.ImageFolder(os.path.join(DATA_DIR, 'val'),   transform=val_tfms)
test_ds  = datasets.ImageFolder(os.path.join(DATA_DIR, 'test'),  transform=val_tfms)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)


idx_to_class = {v:k for k,v in train_ds.class_to_idx.items()}
with open('labels.json','w') as f:
    json.dump(idx_to_class, f)
print(idx_to_class)

In [ ]:
import timm, math, time
import torch.nn as nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import accuracy_score, roc_auc_score

model = timm.create_model('resnet50', pretrained=True, num_classes=len(train_ds.classes))
model = model.to(device)

# Class imbalance handling
# class_counts = torch.tensor([len([1 for p,_ in train_ds.samples if train_ds.classes[p.split(os.sep)[-2]]==c]) for c in train_ds.classes])
# Fallback: compute by directory sizes
class_counts = torch.tensor([len(os.listdir(os.path.join(DATA_DIR, 'train', c))) for c in train_ds.classes])
class_weights = (class_counts.sum() / class_counts).float()
class_weights = class_weights / class_weights.sum() * len(class_weights)
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))

optimizer = AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=10)
scaler = torch.cuda.amp.GradScaler(enabled=(device=='cuda'))

EPOCHS = 10
best_val_auc = -1.0
patience, bad = 3, 0

for epoch in range(1, EPOCHS+1):
    model.train()
    train_losses = []
    y_true_tr, y_pred_tr = [], []

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(device=='cuda')):
            logits = model(imgs)
            loss = criterion(logits, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            train_losses.append(loss.item())
            y_true_tr.extend(labels.detach().cpu().numpy())
            y_pred_tr.extend(torch.softmax(logits,1).detach().cpu().numpy())

    scheduler.step()
    train_acc = accuracy_score(y_true_tr, np.argmax(y_pred_tr,1))
    try:
        train_auc = roc_auc_score(y_true_tr, np.array(y_pred_tr)[:,1]) if len(train_ds.classes)==2 else float('nan')
    except Exception:
        train_auc = float('nan')


    model.eval()
    val_losses = []
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            logits = model(imgs)
            loss = criterion(logits, labels)
            val_losses.append(loss.item())
            y_true.extend(labels.detach().cpu().numpy())
            y_pred.extend(torch.softmax(logits,1).detach().cpu().numpy())

    val_acc = accuracy_score(y_true, np.argmax(y_pred,1))
    try:
        val_auc = roc_auc_score(y_true, np.array(y_pred)[:,1]) if len(train_ds.classes)==2 else float('nan')
    except Exception:
        val_auc = float('nan')

    print(f"Epoch {epoch}: train_loss={np.mean(train_losses):.4f} acc={train_acc:.3f} auc={train_auc:.3f} | val_loss={np.mean(val_losses):.4f} acc={val_acc:.3f} auc={val_auc:.3f}")


    score = val_auc if not math.isnan(val_auc) else val_acc
    if score > best_val_auc:
        best_val_auc = score
        bad = 0
        torch.save(model.state_dict(), 'xray_resnet50.pth')
        print('✔ Saved best model')
    else:
        bad += 1
        if bad >= patience:
            print('⏹ Early stopping')
            break

print('Best validation metric:', best_val_auc)

In [ ]:

from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

model.load_state_dict(torch.load('xray_resnet50.pth', map_location=device))
model.eval()

all_true, all_prob = [], []
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        logits = model(imgs)
        probs = torch.softmax(logits,1).cpu().numpy()
        all_prob.extend(probs)
        all_true.extend(labels.numpy())

pred = np.argmax(np.array(all_prob),1)
print(classification_report(all_true, pred, target_names=train_ds.classes))

cm = confusion_matrix(all_true, pred)
plt.figure()
plt.imshow(cm)
plt.title('Confusion Matrix')
plt.colorbar()
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import cv2
from PIL import Image

target_layers = [model.layer4[-1]]
cam = GradCAM(model=model, target_layers=target_layers)

import torchvision
inv_norm = transforms.Normalize(
    mean=[-m/s for m,s in zip([0.485,0.456,0.406],[0.229,0.224,0.225])],
    std=[1/s for s in [0.229,0.224,0.225]]
)

os.makedirs('cam_samples', exist_ok=True)

count = 0
for img_path, label in test_ds.samples[:8]:
    img = val_tfms(To3Channel()(Image.open(img_path)))
    input_tensor = img.unsqueeze(0).to(device)
    grayscale_cam = cam(input_tensor=input_tensor)[0]

    vis = inv_norm(img).permute(1,2,0).cpu().numpy()
    vis = np.clip(vis,0,1)
    cam_image = show_cam_on_image(vis, grayscale_cam, use_rgb=True)
    out_path = f'cam_samples/{os.path.basename(img_path)}_cam.jpg'
    cv2.imwrite(out_path, cv2.cvtColor(cam_image, cv2.COLOR_RGB2BGR))
    count += 1
print('Saved CAMs to cam_samples/')

In [ ]:
import os, json
assert os.path.exists('xray_resnet50.pth'), 'Model missing!'
assert os.path.exists('labels.json'), 'labels.json missing!'
print('Artifacts ready for serving.')

In [ ]:
torch.save(model.state_dict(), "best_model.pth")
from google.colab import files
files.download("best_model.pth")


In [ ]:
# !pip install gradio

import gradio as gr
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import torchvision.models as models

model = models.resnet50(pretrained=False)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)
model.load_state_dict(torch.load("best_model.pth", map_location="cpu"))
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

def predict(image):
    img = transform(image).unsqueeze(0)
    with torch.no_grad():
        outputs = model(img)
        probs = torch.softmax(outputs, dim=1)[0].tolist()
    return { "Class 0": probs[0], "Class 1": probs[1] }

demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=2),
    title="Medical X-Ray Classifier",
    description="Upload an X-ray image and see diagnosis probabilities"
)

demo.launch(share=True)

You need to provide your ngrok authtoken to `pyngrok`. You can get this from your ngrok dashboard after signing up for a free account.

Add your authtoken to Colab's secrets manager (under the 🔑 icon in the left panel) with the name `NGROK_AUTH_TOKEN`.

Then, add the following code cell to configure `pyngrok`: